In [6]:
import catboost
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import ensemble

import pandas as pd
import numpy as np

In [7]:
pd.options.display.max_columns = None
df = pd.read_csv('../data/hp/train.csv', index_col='Id')
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900


- LotFrontage, LotArea, YearBuilt, YearRemodAdd, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF,
2ndFlrSF, LowQualFinSF, GrLivArea, TotRmsAbvGrd, Fireplaces, KitchenAbvGr, BedroomAbvGr,
FullBath, HalfBath, BsmtFullBath, BsmtHalfBath, GarageYrBlt, GarageCars, GarageArea, OverallQual, 
OverallCond,
WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, MoSold, YrSold - __numbers__

- Alley is weird 

- LotShape, LandContour, Utilities, LotConfig, LandSlope, Negihborhood, Condition1, Condition2, BldgType, HouseStyle, , RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure,
BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, 
KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, 
PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition, Street, MSZoning, MSSubClass - __categoricals__

__SalePrice - the target variable (label)__

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

### now we must fill blanks in columns with less than 1460 values

In [8]:
df["LotFrontage"].fillna(value=0.0, inplace=True)
df["GarageYrBlt"].fillna(value=0.0, inplace=True)
df["MasVnrType"].fillna(value="0", inplace=True)
df["MasVnrArea"].fillna(value=0.0, inplace=True)
df["BsmtQual"].fillna(value="0", inplace=True)

df["BsmtCond"].fillna(value="0", inplace=True)
df["BsmtExposure"].fillna(value="0", inplace=True)
df["BsmtFinType1"].fillna(value="0", inplace=True)
df["BsmtFinType2"].fillna(value="0", inplace=True)
df["Electrical"].fillna(value="0", inplace=True)
df["FireplaceQu"].fillna(value="0", inplace=True)
df["GarageType"].fillna(value="0", inplace=True)
df["GarageFinish"].fillna(value="0", inplace=True)
df["GarageQual"].fillna(value="0", inplace=True)
df["GarageCond"].fillna(value="0", inplace=True)
df["Fence"].fillna(value="0", inplace=True)

In [9]:
# df["Alley"].fillna(value = '0', inplace = True)
# df["PoolQC"].fillna(value = '0', inplace = True)
del df["Alley"] 
del df["PoolQC"]
del df["MiscFeature"]
# delete PoolQC and MiscFeature because they are redundant, possibly delete alley because almost all is NaN

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 78 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 no

In [10]:
CategoricalDataCols = ["LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "Neighborhood", "Condition1", 
                       "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", 
                       "MasVnrType", "MasVnrArea", "ExterQual", "ExterCond", "Foundation", "BsmtQual", 
                       "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", 
                       "CentralAir", "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", 
                       "GarageFinish", "GarageQual", "GarageCond", "PavedDrive", "Fence", "SaleType", 
                       "SaleCondition", "Street", "MSZoning", "MSSubClass"]

In [11]:
Labels = df["SalePrice"]
Data = df.drop("SalePrice", axis=1)


In [34]:
train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
655,20,RL,91.0,10437,Pave,0,IR1,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,1Story,8,6,1995,1995,Hip,CompShg,MetalSd,MetalSd,BrkFace,660.0,Gd,Gd,PConc,Gd,TA,Gd,GLQ,1696,Unf,0,413,2109,GasA,Ex,Y,SBrkr,2113,0,0,2113,1,0,2,1,2,1,Gd,7,Typ,1,TA,Attchd,1995.0,Fin,3,839,TA,TA,Y,236,46,0,0,0,0,0,0,8,2008,WD,Normal
233,160,RM,21.0,1680,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,BrDale,Norm,Norm,Twnhs,2Story,6,5,1972,1972,Gable,CompShg,HdBoard,HdBoard,BrkFace,297.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,483,483,GasA,TA,Y,SBrkr,483,504,0,987,0,0,1,1,2,1,TA,5,Typ,1,Po,Attchd,1972.0,Unf,1,288,TA,TA,Y,0,0,0,0,0,0,0,0,6,2006,WD,Normal
302,60,RL,66.0,16226,Pave,0,IR3,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,8,5,1998,1999,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,281,Unf,0,747,1028,GasA,Ex,Y,SBrkr,1210,1242,0,2452,0,0,2,1,4,1,Gd,9,Typ,1,TA,BuiltIn,1998.0,Fin,2,683,TA,TA,Y,208,50,0,0,0,0,0,0,5,2007,WD,Normal
1395,120,RL,53.0,4045,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2006,2006,Hip,CompShg,VinylSd,VinylSd,BrkFace,45.0,Gd,TA,PConc,Gd,TA,Av,GLQ,1070,Unf,0,286,1356,GasA,Ex,Y,SBrkr,1500,0,0,1500,1,0,2,0,2,1,Gd,6,Typ,1,Gd,Attchd,2006.0,Fin,3,648,TA,TA,Y,161,20,0,0,0,0,0,0,10,2006,New,Partial
1384,30,RL,0.0,25339,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,7,1918,2007,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,Gd,BrkTil,TA,TA,No,Unf,0,Unf,0,816,816,GasA,Ex,Y,SBrkr,1416,0,0,1416,0,0,2,0,3,1,Gd,7,Typ,0,0,Attchd,2007.0,Unf,2,576,TA,TA,N,0,0,112,0,0,0,0,0,8,2007,WD,Normal
1338,30,RM,153.0,4118,Pave,Grvl,IR1,Bnk,AllPub,Corner,Mod,OldTown,Feedr,Norm,1Fam,1Story,4,4,1941,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,693,693,Grav,Fa,N,FuseA,693,0,0,693,0,0,1,0,2,1,Fa,4,Typ,0,0,0,0.0,0,0,0,0,0,N,0,20,0,0,0,0,0,0,3,2006,WD,Normal
285,120,RL,50.0,8012,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,TwnhsE,1Story,6,5,1992,1992,Gable,CompShg,Plywood,ImStucc,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,430,Unf,0,1145,1575,GasA,Gd,Y,SBrkr,1575,0,0,1575,1,0,2,0,2,1,Gd,5,Typ,0,0,Attchd,1992.0,RFn,2,529,TA,TA,Y,0,0,52,0,0,0,0,0,7,2007,WD,Normal
1024,120,RL,43.0,3182,Pave,0,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,7,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,14.0,Gd,TA,PConc,Gd,Gd,No,GLQ,16,Unf,0,1330,1346,GasA,Ex,Y,SBrkr,1504,0,0,1504,0,0,2,0,2,1,Gd,7,Typ,1,Gd,Attchd,2005.0,Fin,2,437,TA,TA,Y,156,20,0,0,0,0,0,0,5,2008,WD,Normal
738,60,RL,72.0,10463,Pave,0,IR1,HLS,AllPub,CulDSac,Gtl,Gilbert,Norm,Norm,1Fam,2Story,8,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,893,893,GasA,Ex,Y,SBrkr,901,900,0,1801,0,0,2,1,3,1,Gd,8,Typ,1,Gd,Attchd,2005.0,Fin,3,800,TA,TA,Y,0,116,0,0,0,0,0,0,6,2006,WD,Normal


Now let's one-hot encode all the categoricals

In [12]:
for col in CategoricalDataCols:
    Data[col] = pd.Categorical(Data[col])

train_data, test_data, train_labels, test_labels = train_test_split(Data, Labels, random_state=98987, test_size=0.25)

In [13]:
train_data_cats = train_data[CategoricalDataCols]
test_data_cats = test_data[CategoricalDataCols]
oh_train_data = train_data
for col in CategoricalDataCols:
    oh_train_data = oh_train_data.drop(col, axis=1)
oh_test_data = test_data
for col in CategoricalDataCols:
    oh_test_data = oh_test_data.drop(col, axis=1)

display(oh_test_data.head())

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
883,0.0,9636,6,5,1992,1993,0,0,808,808,808,785,0,1593,0,0,2,1,3,1,7,1,1993.0,2,389,342,40,0,0,0,0,0,12,2009
1443,85.0,11003,10,5,2008,2008,765,0,252,1017,1026,981,0,2007,1,0,2,1,3,1,10,1,2008.0,3,812,168,52,0,0,0,0,0,4,2009
1136,60.0,6180,6,5,1926,1950,0,0,960,960,986,0,0,986,0,0,1,0,2,1,5,1,1926.0,1,180,0,128,0,0,0,0,0,5,2007
631,50.0,9000,5,6,1880,1991,0,0,636,636,1089,661,0,1750,0,0,1,0,3,1,8,0,1937.0,1,240,0,0,293,0,0,0,0,6,2006
1067,59.0,7837,6,7,1993,1994,0,0,799,799,799,772,0,1571,0,0,2,1,3,1,7,1,1993.0,2,380,0,40,0,0,0,0,0,5,2009


In [58]:
display(train_data_cats.head())

,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition,Street,MSZoning,MSSubClass
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
655,IR1,Lvl,AllPub,Inside,Gtl,NoRidge,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,BrkFace,660.0,Gd,Gd,PConc,Gd,TA,Gd,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,Fin,TA,TA,Y,0,WD,Normal,Pave,RL,20
233,Reg,Lvl,AllPub,Inside,Gtl,BrDale,Norm,Norm,Twnhs,2Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,297.0,TA,TA,CBlock,TA,TA,No,Unf,Unf,GasA,TA,Y,SBrkr,TA,Typ,Po,Attchd,Unf,TA,TA,Y,0,WD,Normal,Pave,RM,160
302,IR3,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,BuiltIn,Fin,TA,TA,Y,0,WD,Normal,Pave,RL,60
1395,Reg,Lvl,AllPub,Inside,Gtl,Blmngtn,Norm,Norm,TwnhsE,1Story,Hip,CompShg,VinylSd,VinylSd,BrkFace,45.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,0,New,Partial,Pave,RL,120
1384,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,Gd,BrkTil,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,0,Attchd,Unf,TA,TA,N,0,WD,Normal,Pave,RL,30


In [57]:
display(test_data_cats.head())

,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,Fence,SaleType,SaleCondition,Street,MSZoning,MSSubClass
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
883,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,BuiltIn,RFn,TA,TA,Y,MnPrv,WD,Normal,Pave,RL,60
1443,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,Stone,160.0,Ex,TA,PConc,Ex,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Ex,Typ,Ex,Attchd,Fin,TA,TA,Y,0,WD,Normal,Pave,FV,60
1136,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,Unf,GasA,TA,N,SBrkr,TA,Typ,Gd,Detchd,Unf,TA,TA,Y,0,WD,Normal,Pave,RM,30
631,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Artery,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,BrkTil,Fa,Fa,No,Unf,Unf,GasA,TA,Y,FuseA,Ex,Typ,0,Detchd,Unf,Fa,Po,N,MnPrv,WD,Abnorml,Pave,RM,70
1067,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,0,WD,Normal,Pave,RL,60


In [14]:
one_hot = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
train_data_cats = one_hot.fit_transform(train_data_cats).toarray()

In [15]:
test_data_cats = one_hot.transform(test_data_cats).toarray()

In [16]:
for i in range(len(train_data_cats[0])):
#     print(len(train_data_cats))
#     print(len(train_data_cats[0]))
#     print
#     print(i)
    oh_train_data['one-hot' + str(i)] = train_data_cats[:, i]
    
for i in range(len(test_data_cats[0])):
    oh_test_data['one-hot' + str(i)] = test_data_cats[:, i]

display(oh_train_data.head())
display(oh_train_data.head())

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,one-hot0,one-hot1,one-hot2,one-hot3,one-hot4,one-hot5,one-hot6,one-hot7,one-hot8,one-hot9,one-hot10,one-hot11,one-hot12,one-hot13,one-hot14,one-hot15,one-hot16,one-hot17,one-hot18,one-hot19,one-hot20,one-hot21,one-hot22,one-hot23,one-hot24,one-hot25,one-hot26,one-hot27,one-hot28,one-hot29,one-hot30,one-hot31,one-hot32,one-hot33,one-hot34,one-hot35,one-hot36,one-hot37,one-hot38,one-hot39,one-hot40,one-hot41,one-hot42,one-hot43,one-hot44,one-hot45,one-hot46,one-hot47,one-hot48,one-hot49,one-hot50,one-hot51,one-hot52,one-hot53,one-hot54,one-hot55,one-hot56,one-hot57,one-hot58,one-hot59,one-hot60,one-hot61,one-hot62,one-hot63,one-hot64,one-hot65,one-hot66,one-hot67,one-hot68,one-hot69,one-hot70,one-hot71,one-hot72,one-hot73,one-hot74,one-hot75,one-hot76,one-hot77,one-hot78,one-hot79,one-hot80,one-hot81,one-hot82,one-hot83,one-hot84,one-hot85,one-hot86,one-hot87,one-hot88,one-hot89,one-hot90,one-hot91,one-hot92,one-hot93,one-hot94,one-hot95,one-hot96,one-hot97,one-hot98,one-hot99,one-hot100,one-hot101,one-hot102,one-hot103,one-hot104,one-hot105,one-hot106,one-hot107,one-hot108,one-hot109,one-hot110,one-hot111,one-hot112,one-hot113,one-hot114,one-hot115,one-hot116,one-hot117,one-hot118,one-hot119,one-hot120,one-hot121,one-hot122,one-hot123,one-hot124,one-hot125,one-hot126,one-hot127,one-hot128,one-hot129,one-hot130,one-hot131,one-hot132,one-hot133,one-hot134,one-hot135,one-hot136,one-hot137,one-hot138,one-hot139,one-hot140,one-hot141,one-hot142,one-hot143,one-hot144,one-hot145,one-hot146,one-hot147,one-hot148,one-hot149,one-hot150,one-hot151,one-hot152,one-hot153,one-hot154,one-hot155,one-hot156,one-hot157,one-hot158,one-hot159,one-hot160,one-hot161,one-hot162,one-hot163,one-hot164,one-hot165,one-hot166,one-hot167,one-hot168,one-hot169,one-hot170,one-hot171,one-hot172,one-hot173,one-hot174,one-hot175,one-hot176,one-hot177,one-hot178,one-hot179,one-hot180,one-hot181,one-hot182,one-hot183,one-hot184,one-hot185,one-hot186,one-hot187,one-hot188,one-hot189,one-hot190,one-hot191,one-hot192,one-hot193,one-hot194,one-hot195,one-hot196,one-hot197,one-hot198,one-hot199,one-hot200,one-hot201,one-hot202,one-hot203,one-hot204,one-hot205,one-hot206,one-hot207,one-hot208,one-hot209,one-hot210,one-hot211,one-hot212,one-hot213,one-hot214,one-hot215,one-hot216,one-hot217,one-hot218,one-hot219,one-hot220,one-hot221,one-hot222,one-hot223,one-hot224,one-hot225,one-hot226,one-hot227,one-hot228,one-hot229,one-hot230,one-hot231,one-hot232,one-hot233,one-hot234,one-hot235,one-hot236,one-hot237,one-hot238,one-hot239,one-hot240,one-hot241,one-hot242,one-hot243,one-hot244,one-hot245,one-hot246,one-hot247,one-hot248,one-hot249,one-hot250,one-hot251,one-hot252,one-hot253,one-hot254,one-hot255,one-hot256,one-hot257,one-hot258,one-hot259,one-hot260,one-hot261,one-hot262,one-hot263,one-hot264,one-hot265,one-hot266,one-hot267,one-hot268,one-hot269,one-hot270,one-hot271,one-hot272,one-hot273,one-hot274,one-hot275,one-hot276,one-hot277,one-hot278,one-hot279,one-hot280,one-hot281,one-hot282,one-hot283,one-hot284,one-hot285,one-hot286,one-hot287,one-hot288,one-hot289,one-hot290,one-hot291,one-hot292,one-hot293,one-hot294,one-hot295,one-hot296,one-hot297,one-hot298,one-hot299,one-hot300,one-hot301,one-hot302,one-hot303,one-hot304,one-hot305,one-hot306,one-hot307,one-hot308,one-hot309,one-hot310,one-hot311,one-hot312,one-hot313,one-hot314,one-hot315,one-hot316,one-hot317,one-hot318,one-hot319,one-hot320,one-hot321,one-hot322,one-hot323,one-hot324,one-hot325,one-hot326,one-hot327,one-hot328,one-hot329,one-hot330,one-hot331,one-hot332,one-hot333,one-hot334,one-hot335,one-hot336,one-hot337,one-hot338,one-hot339

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,one-hot0,one-hot1,one-hot2,one-hot3,one-hot4,one-hot5,one-hot6,one-hot7,one-hot8,one-hot9,one-hot10,one-hot11,one-hot12,one-hot13,one-hot14,one-hot15,one-hot16,one-hot17,one-hot18,one-hot19,one-hot20,one-hot21,one-hot22,one-hot23,one-hot24,one-hot25,one-hot26,one-hot27,one-hot28,one-hot29,one-hot30,one-hot31,one-hot32,one-hot33,one-hot34,one-hot35,one-hot36,one-hot37,one-hot38,one-hot39,one-hot40,one-hot41,one-hot42,one-hot43,one-hot44,one-hot45,one-hot46,one-hot47,one-hot48,one-hot49,one-hot50,one-hot51,one-hot52,one-hot53,one-hot54,one-hot55,one-hot56,one-hot57,one-hot58,one-hot59,one-hot60,one-hot61,one-hot62,one-hot63,one-hot64,one-hot65,one-hot66,one-hot67,one-hot68,one-hot69,one-hot70,one-hot71,one-hot72,one-hot73,one-hot74,one-hot75,one-hot76,one-hot77,one-hot78,one-hot79,one-hot80,one-hot81,one-hot82,one-hot83,one-hot84,one-hot85,one-hot86,one-hot87,one-hot88,one-hot89,one-hot90,one-hot91,one-hot92,one-hot93,one-hot94,one-hot95,one-hot96,one-hot97,one-hot98,one-hot99,one-hot100,one-hot101,one-hot102,one-hot103,one-hot104,one-hot105,one-hot106,one-hot107,one-hot108,one-hot109,one-hot110,one-hot111,one-hot112,one-hot113,one-hot114,one-hot115,one-hot116,one-hot117,one-hot118,one-hot119,one-hot120,one-hot121,one-hot122,one-hot123,one-hot124,one-hot125,one-hot126,one-hot127,one-hot128,one-hot129,one-hot130,one-hot131,one-hot132,one-hot133,one-hot134,one-hot135,one-hot136,one-hot137,one-hot138,one-hot139,one-hot140,one-hot141,one-hot142,one-hot143,one-hot144,one-hot145,one-hot146,one-hot147,one-hot148,one-hot149,one-hot150,one-hot151,one-hot152,one-hot153,one-hot154,one-hot155,one-hot156,one-hot157,one-hot158,one-hot159,one-hot160,one-hot161,one-hot162,one-hot163,one-hot164,one-hot165,one-hot166,one-hot167,one-hot168,one-hot169,one-hot170,one-hot171,one-hot172,one-hot173,one-hot174,one-hot175,one-hot176,one-hot177,one-hot178,one-hot179,one-hot180,one-hot181,one-hot182,one-hot183,one-hot184,one-hot185,one-hot186,one-hot187,one-hot188,one-hot189,one-hot190,one-hot191,one-hot192,one-hot193,one-hot194,one-hot195,one-hot196,one-hot197,one-hot198,one-hot199,one-hot200,one-hot201,one-hot202,one-hot203,one-hot204,one-hot205,one-hot206,one-hot207,one-hot208,one-hot209,one-hot210,one-hot211,one-hot212,one-hot213,one-hot214,one-hot215,one-hot216,one-hot217,one-hot218,one-hot219,one-hot220,one-hot221,one-hot222,one-hot223,one-hot224,one-hot225,one-hot226,one-hot227,one-hot228,one-hot229,one-hot230,one-hot231,one-hot232,one-hot233,one-hot234,one-hot235,one-hot236,one-hot237,one-hot238,one-hot239,one-hot240,one-hot241,one-hot242,one-hot243,one-hot244,one-hot245,one-hot246,one-hot247,one-hot248,one-hot249,one-hot250,one-hot251,one-hot252,one-hot253,one-hot254,one-hot255,one-hot256,one-hot257,one-hot258,one-hot259,one-hot260,one-hot261,one-hot262,one-hot263,one-hot264,one-hot265,one-hot266,one-hot267,one-hot268,one-hot269,one-hot270,one-hot271,one-hot272,one-hot273,one-hot274,one-hot275,one-hot276,one-hot277,one-hot278,one-hot279,one-hot280,one-hot281,one-hot282,one-hot283,one-hot284,one-hot285,one-hot286,one-hot287,one-hot288,one-hot289,one-hot290,one-hot291,one-hot292,one-hot293,one-hot294,one-hot295,one-hot296,one-hot297,one-hot298,one-hot299,one-hot300,one-hot301,one-hot302,one-hot303,one-hot304,one-hot305,one-hot306,one-hot307,one-hot308,one-hot309,one-hot310,one-hot311,one-hot312,one-hot313,one-hot314,one-hot315,one-hot316,one-hot317,one-hot318,one-hot319,one-hot320,one-hot321,one-hot322,one-hot323,one-hot324,one-hot325,one-hot326,one-hot327,one-hot328,one-hot329,one-hot330,one-hot331,one-hot332,one-hot333,one-hot334,one-hot335,one-hot336,one-hot337,one-hot338,one-hot339

In [81]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

sklearn_boost.fit(oh_train_data, train_labels)
print("train score", sklearn_boost.score(oh_train_data, train_labels))
print("test score", sklearn_boost.score(oh_test_data, test_labels))

train score 0.9725603672039187
test score 0.8420232319410241


In [83]:
cboost = catboost.CatBoostRegressor(iterations=None,
                        learning_rate=None,
                        depth=None,
                        l2_leaf_reg=None,
                        model_size_reg=None,
                        rsm=None,
                        loss_function='RMSE',
                        border_count=None,
                        feature_border_type=None,
                        per_float_feature_quantization=None,
                        input_borders=None,
                        output_borders=None,
                        fold_permutation_block=None,
                        od_pval=None,
                        od_wait=None,
                        od_type=None,
                        nan_mode=None,
                        counter_calc_method=None,
                        leaf_estimation_iterations=None,
                        leaf_estimation_method=None,
                        thread_count=None,
                        random_seed=None,
                        use_best_model=None,
                        best_model_min_trees=None,
                        verbose=False,
                        silent=None,
                        logging_level=None,
                        metric_period=None,
                        ctr_leaf_count_limit=None,
                        store_all_simple_ctr=None,
                        max_ctr_complexity=None,
                        has_time=None,
                        allow_const_label=None,
                        one_hot_max_size=None,
                        random_strength=None,
                        name=None,
                        ignored_features=None,
                        train_dir=None,
                        custom_metric=None,
                        eval_metric=None,
                        bagging_temperature=None,
                        save_snapshot=None,
                        snapshot_file=None,
                        snapshot_interval=None,
                        fold_len_multiplier=None,
                        used_ram_limit=None,
                        gpu_ram_part=None,
                        pinned_memory_size=None,
                        allow_writing_files=None,
                        final_ctr_computation_mode=None,
                        approx_on_full_history=None,
                        boosting_type=None,
                        simple_ctr=None,
                        combinations_ctr=None,
                        per_feature_ctr=None,
                        ctr_target_border_count=None,
                        task_type=None,
                        device_config=None,                        
                        devices=None,
                        bootstrap_type=None,
                        subsample=None,                        
                        sampling_unit=None,
                        dev_score_calc_obj_block_size=None,
                        max_depth=None,
                        n_estimators=None,
                        num_boost_round=None,
                        num_trees=None,
                        colsample_bylevel=None,
                        random_state=None,
                        reg_lambda=None,
                        objective=None,
                        eta=None,
                        max_bin=None,
                        gpu_cat_features_storage=None,
                        data_partition=None,
                        metadata=None,
                        early_stopping_rounds=None,
                        cat_features=None,
                        grow_policy=None,
                        min_data_in_leaf=None,
                        min_child_samples=None,
                        max_leaves=None,
                        num_leaves=None,
                        score_function=None,
                        leaf_estimation_backtracking=None,
                        ctr_history_unit=None,
                        monotone_constraints=None)
cboost.fit(oh_train_data, train_labels)
print("test score", cboost.score(oh_test_data, test_labels))
print("train score", cboost.score(oh_train_data, train_labels))

test score 0.8472655334171177
train score 0.9895222095429762


In [89]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


params = {
          'max_depth': [3, 5 ,10, 2],
          'max_features': [None, 'sqrt'],

          }

GradBoostGrid = sklearn.model_selection.GridSearchCV(sklearn_boost, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)
GradBoostGrid.fit(oh_train_data, train_labels)

display(pd.DataFrame(GradBoostGrid.cv_results_).sort_values('rank_test_score').head())

print("test score", GradBoostGrid.score(oh_test_data, test_labels))
print("train score", GradBoostGrid.score(oh_train_data, train_labels))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.082211,0.000797,0.001994,1.266988e-06,5,sqrt,"{'max_depth': 5, 'max_features': 'sqrt'}",0.859416,0.881565,0.905737,0.753564,0.933711,0.866799,0.061787,1
1,0.057471,0.000484,0.001996,1.818245e-06,3,sqrt,"{'max_depth': 3, 'max_features': 'sqrt'}",0.869393,0.891446,0.890312,0.738320,0.930508,0.863996,0.065869,2
0,0.756243,0.007716,0.002001,1.191750e-05,3,None,"{'max_depth': 3, 'max_features': None}",0.886059,0.905024,0.891479,0.696342,0.933899,0.862561,0.084746,3
2,1.191184,0.005688,0.001998,5.631122e-06,5,None,"{'max_depth': 5, 'max_features': None}",0.871143,0.897761,0.890684,0.721687,0.916377,0.859530,0.070426,4
7,0.046077,0.000399,0.001995,1.907349e-07,2,sqrt,"{'max_depth': 2, 'max_features': 'sqrt'}",0.855777,0.890400,0.882699,0.742561,0.913935,0.857075,0.060192,5


test score 0.8087165180595882
train score 0.9825962641169265


#### now i will normalize data

In [90]:
minmax = sklearn.preprocessing.MinMaxScaler()
oh_train_data = minmax.fit_transform(oh_train_data)
oh_test_data = minmax.transform(oh_test_data)

In [92]:
display(oh_test_data)

array([[0.        , 0.0510284 , 0.55555556, ..., 0.        , 0.        ,
        0.        ],
       [0.2715655 , 0.05939643, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.19169329, 0.02987267, 0.55555556, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.33546326, 0.04389079, 0.22222222, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.19076273, 0.33333333, ..., 0.        , 0.        ,
        1.        ],
       [0.23961661, 0.05402179, 0.55555556, ..., 0.        , 0.        ,
        0.        ]])

In [93]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

sklearn_boost.fit(oh_train_data, train_labels)
print("train score", sklearn_boost.score(oh_train_data, train_labels))
print("test score", sklearn_boost.score(oh_test_data, test_labels))

train score 0.9725603672039187
test score 0.8419843008677319


In [94]:
cboost = catboost.CatBoostRegressor(iterations=None,
                        learning_rate=None,
                        depth=None,
                        l2_leaf_reg=None,
                        model_size_reg=None,
                        rsm=None,
                        loss_function='RMSE',
                        border_count=None,
                        feature_border_type=None,
                        per_float_feature_quantization=None,
                        input_borders=None,
                        output_borders=None,
                        fold_permutation_block=None,
                        od_pval=None,
                        od_wait=None,
                        od_type=None,
                        nan_mode=None,
                        counter_calc_method=None,
                        leaf_estimation_iterations=None,
                        leaf_estimation_method=None,
                        thread_count=None,
                        random_seed=None,
                        use_best_model=None,
                        best_model_min_trees=None,
                        verbose=False,
                        silent=None,
                        logging_level=None,
                        metric_period=None,
                        ctr_leaf_count_limit=None,
                        store_all_simple_ctr=None,
                        max_ctr_complexity=None,
                        has_time=None,
                        allow_const_label=None,
                        one_hot_max_size=None,
                        random_strength=None,
                        name=None,
                        ignored_features=None,
                        train_dir=None,
                        custom_metric=None,
                        eval_metric=None,
                        bagging_temperature=None,
                        save_snapshot=None,
                        snapshot_file=None,
                        snapshot_interval=None,
                        fold_len_multiplier=None,
                        used_ram_limit=None,
                        gpu_ram_part=None,
                        pinned_memory_size=None,
                        allow_writing_files=None,
                        final_ctr_computation_mode=None,
                        approx_on_full_history=None,
                        boosting_type=None,
                        simple_ctr=None,
                        combinations_ctr=None,
                        per_feature_ctr=None,
                        ctr_target_border_count=None,
                        task_type=None,
                        device_config=None,                        
                        devices=None,
                        bootstrap_type=None,
                        subsample=None,                        
                        sampling_unit=None,
                        dev_score_calc_obj_block_size=None,
                        max_depth=None,
                        n_estimators=None,
                        num_boost_round=None,
                        num_trees=None,
                        colsample_bylevel=None,
                        random_state=None,
                        reg_lambda=None,
                        objective=None,
                        eta=None,
                        max_bin=None,
                        gpu_cat_features_storage=None,
                        data_partition=None,
                        metadata=None,
                        early_stopping_rounds=None,
                        cat_features=None,
                        grow_policy=None,
                        min_data_in_leaf=None,
                        min_child_samples=None,
                        max_leaves=None,
                        num_leaves=None,
                        score_function=None,
                        leaf_estimation_backtracking=None,
                        ctr_history_unit=None,
                        monotone_constraints=None)
cboost.fit(oh_train_data, train_labels)
print("test score", cboost.score(oh_test_data, test_labels))
print("train score", cboost.score(oh_train_data, train_labels))

test score 0.8472753135866607
train score 0.9895222095429762


In [95]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


params = {
          'max_depth': [3, 5 ,10, 2],
          'max_features': [None, 'sqrt'],

          }

GradBoostGrid = sklearn.model_selection.GridSearchCV(sklearn_boost, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)
GradBoostGrid.fit(oh_train_data, train_labels)

display(pd.DataFrame(GradBoostGrid.cv_results_).sort_values('rank_test_score').head())

print("test score", GradBoostGrid.score(oh_test_data, test_labels))
print("train score", GradBoostGrid.score(oh_train_data, train_labels))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.091898,0.001214,0.001402,0.000483,5,sqrt,"{'max_depth': 5, 'max_features': 'sqrt'}",0.859287,0.881422,0.905797,0.753730,0.933624,0.866772,0.061711,1
1,0.062463,0.000815,0.001795,0.000399,3,sqrt,"{'max_depth': 3, 'max_features': 'sqrt'}",0.869403,0.891471,0.890312,0.738320,0.930613,0.864024,0.065892,2
0,0.909186,0.018882,0.001197,0.000399,3,None,"{'max_depth': 3, 'max_features': None}",0.885818,0.905024,0.891312,0.696342,0.933899,0.862479,0.084721,3
2,1.392431,0.007597,0.001388,0.000496,5,None,"{'max_depth': 5, 'max_features': None}",0.871131,0.897803,0.890544,0.721912,0.916437,0.859565,0.070340,4
7,0.049455,0.001339,0.001003,0.000013,2,sqrt,"{'max_depth': 2, 'max_features': 'sqrt'}",0.855804,0.890492,0.882699,0.742588,0.913935,0.857104,0.060192,5


test score 0.8083198428149783
train score 0.9825962641169265


__there should be xgboost and lightgbm here, but later.__
#### now with PCA

In [97]:
from sklearn.decomposition import PCA
pcator = PCA(n_components=0.9, copy=True, whiten=False, svd_solver='full', tol=0.0, iterated_power='auto', random_state=None)
## MUST USE FIT_TRANSFORM, will break otherwise because of copy=false
oh_train_data_reduced = pcator.fit_transform(oh_train_data)
print("done fitting")
oh_test_data_reduced = pcator.transform(oh_test_data)

done fitting


In [98]:
cboost = catboost.CatBoostRegressor(iterations=None,
                        learning_rate=None,
                        depth=None,
                        l2_leaf_reg=None,
                        model_size_reg=None,
                        rsm=None,
                        loss_function='RMSE',
                        border_count=None,
                        feature_border_type=None,
                        per_float_feature_quantization=None,
                        input_borders=None,
                        output_borders=None,
                        fold_permutation_block=None,
                        od_pval=None,
                        od_wait=None,
                        od_type=None,
                        nan_mode=None,
                        counter_calc_method=None,
                        leaf_estimation_iterations=None,
                        leaf_estimation_method=None,
                        thread_count=None,
                        random_seed=None,
                        use_best_model=None,
                        best_model_min_trees=None,
                        verbose=False,
                        silent=None,
                        logging_level=None,
                        metric_period=None,
                        ctr_leaf_count_limit=None,
                        store_all_simple_ctr=None,
                        max_ctr_complexity=None,
                        has_time=None,
                        allow_const_label=None,
                        one_hot_max_size=None,
                        random_strength=None,
                        name=None,
                        ignored_features=None,
                        train_dir=None,
                        custom_metric=None,
                        eval_metric=None,
                        bagging_temperature=None,
                        save_snapshot=None,
                        snapshot_file=None,
                        snapshot_interval=None,
                        fold_len_multiplier=None,
                        used_ram_limit=None,
                        gpu_ram_part=None,
                        pinned_memory_size=None,
                        allow_writing_files=None,
                        final_ctr_computation_mode=None,
                        approx_on_full_history=None,
                        boosting_type=None,
                        simple_ctr=None,
                        combinations_ctr=None,
                        per_feature_ctr=None,
                        ctr_target_border_count=None,
                        task_type=None,
                        device_config=None,                        
                        devices=None,
                        bootstrap_type=None,
                        subsample=None,                        
                        sampling_unit=None,
                        dev_score_calc_obj_block_size=None,
                        max_depth=None,
                        n_estimators=None,
                        num_boost_round=None,
                        num_trees=None,
                        colsample_bylevel=None,
                        random_state=None,
                        reg_lambda=None,
                        objective=None,
                        eta=None,
                        max_bin=None,
                        gpu_cat_features_storage=None,
                        data_partition=None,
                        metadata=None,
                        early_stopping_rounds=None,
                        cat_features=None,
                        grow_policy=None,
                        min_data_in_leaf=None,
                        min_child_samples=None,
                        max_leaves=None,
                        num_leaves=None,
                        score_function=None,
                        leaf_estimation_backtracking=None,
                        ctr_history_unit=None,
                        monotone_constraints=None)
cboost.fit(oh_train_data_reduced, train_labels)
print("test score", cboost.score(oh_test_data_reduced, test_labels))
print("train score", cboost.score(oh_train_data_reduced, train_labels))

test score 0.7474930922954502
train score 0.9954828054465529


In [108]:
sklearn_boost = ensemble.GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, 
                                   subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                   min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                   random_state=98987, max_features=None, alpha=0.9, verbose=0, 
                                   max_leaf_nodes=None, warm_start=False, presort='deprecated', 
                                   validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


params = {
          'max_depth': [3, 5, 2, 7],
          'max_features': [None, 'sqrt', 'log2'],

          }

GradBoostGrid = sklearn.model_selection.GridSearchCV(sklearn_boost, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)
GradBoostGrid.fit(oh_train_data_reduced, train_labels)

display(pd.DataFrame(GradBoostGrid.cv_results_).sort_values('rank_test_score').head())

print("test score", GradBoostGrid.score(oh_test_data_reduced, test_labels))
print("train score", GradBoostGrid.score(oh_train_data_reduced, train_labels))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,2.122795,0.004948,0.000997,0.000003,5,None,"{'max_depth': 5, 'max_features': None}",0.730183,0.717999,0.814010,0.622721,0.806964,0.738376,0.069690,1
0,1.359100,0.011802,0.000998,0.000002,3,None,"{'max_depth': 3, 'max_features': None}",0.733401,0.729272,0.791528,0.628052,0.804840,0.737419,0.062477,2
4,0.260013,0.002833,0.001596,0.000489,5,sqrt,"{'max_depth': 5, 'max_features': 'sqrt'}",0.680057,0.685422,0.767564,0.714332,0.794398,0.728355,0.045324,3
1,0.167567,0.001810,0.001004,0.000012,3,sqrt,"{'max_depth': 3, 'max_features': 'sqrt'}",0.665928,0.721603,0.769828,0.664166,0.799086,0.724122,0.054215,4
5,0.181717,0.001593,0.001197,0.000399,5,log2,"{'max_depth': 5, 'max_features': 'log2'}",0.664853,0.694187,0.743121,0.688031,0.799366,0.717912,0.048044,5


test score 0.6853514520595103
train score 0.9962506901672723


#### PCA makes everything significantly worse

In [25]:
import xgboost as xgb

In [113]:
xgboost = xgb.XGBRegressor()
xgboost.fit(oh_train_data, train_labels)
xgboost.score(oh_test_data, test_labels)

d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:37:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.845356133220309

In [2]:
import lightgbm as lgb

In [18]:
lgbreg = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=-1, 
                           learning_rate=0.1, n_estimators=100, subsample_for_bin=200000,
                           objective=None, class_weight=None, min_split_gain=0.0, 
                           min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                           subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, 
                           reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, importance_type='split')

lgbreg.fit(oh_train_data, train_labels)
print(lgbreg.score(oh_test_data, test_labels))
print(lgbreg.score(oh_train_data, train_labels))

0.8299393997016506
0.9811004059839733


In [24]:
params = {
    'boosting_type': ['gbdt'],
    'max_depth': [-1, 1, 3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.5, 1.],
    'n_estimators': [20, 50, 100],
}


lgbgrid = sklearn.model_selection.GridSearchCV(lgbreg, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)

lgbgrid.fit(oh_train_data, train_labels)

display(pd.DataFrame(lgbgrid.cv_results_).sort_values('rank_test_score').head())
print('train score', lgbgrid.score(oh_train_data, train_labels))
print('test score', lgbgrid.score(oh_test_data, test_labels))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,0.157976,0.003817,0.107715,0.001260,gbdt,0.1,3,100,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.863156,0.917473,0.882947,0.777665,0.935346,0.875317,0.054996,1
11,0.177727,0.005688,0.110907,0.001932,gbdt,0.1,5,100,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.865608,0.911858,0.874206,0.780643,0.932810,0.873025,0.052294,2
17,0.261706,0.006807,0.109707,0.002092,gbdt,0.05,-1,100,"{'boosting_type': 'gbdt', 'learning_rate': 0.0...",0.848908,0.912025,0.876280,0.794514,0.925262,0.871398,0.046889,3
1,0.197673,0.004649,0.111108,0.001850,gbdt,0.1,-1,50,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.849375,0.913861,0.873721,0.793797,0.926106,0.871372,0.047557,4
29,0.224003,0.006203,0.115292,0.001620,gbdt,0.05,7,100,"{'boosting_type': 'gbdt', 'learning_rate': 0.0...",0.847582,0.912490,0.876316,0.795709,0.922497,0.870919,0.046084,5


train score 0.9527467409844161
test score 0.8285823618167415


In [28]:
xgboost = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, 
                            verbosity=1, objective='reg:squarederror', 
                            booster='gbtree', tree_method='auto', n_jobs=1, 
                            gamma=0, min_child_weight=1, max_delta_step=0, 
                            subsample=1, colsample_bytree=1, colsample_bylevel=1, 
                            colsample_bynode=1, reg_alpha=0, reg_lambda=1, 
                            scale_pos_weight=1, base_score=0.5, 
                            random_state=1998, missing=None, num_parallel_tree=1, 
                            importance_type='gain')


params = {
    'max_depth': [1, 3, 5, 10, 20],
    'learning_rate': [0.1, 0.001, 0.5], 
    'n_estimators': [100, 50]
}


xgrid = sklearn.model_selection.GridSearchCV(xgboost, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)

xgrid.fit(oh_train_data, train_labels)

display(pd.DataFrame(xgrid.cv_results_).sort_values('rank_test_score').head())
print('train score', xgrid.score(oh_train_data, train_labels))
print('test score', xgrid.score(oh_test_data, test_labels))

d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
d:\utilites\programming\anaconda-python\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future vers

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,2.074148,0.036912,0.009791,0.001166,0.1,5,100,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.880136,0.889462,0.883297,0.752627,0.923885,0.865881,0.058743,1
5,1.046822,0.022761,0.008784,0.000747,0.1,5,50,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",0.874474,0.883075,0.879905,0.757372,0.916890,0.862343,0.054552,2
2,1.420881,0.015894,0.008977,0.000631,0.1,3,100,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.872850,0.897860,0.881375,0.696956,0.931173,0.856043,0.082007,3
7,1.726460,0.013507,0.008981,0.000003,0.1,10,50,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.856949,0.878537,0.868324,0.731488,0.913991,0.849858,0.062190,4
9,2.413032,0.028376,0.008977,0.000003,0.1,20,50,"{'learning_rate': 0.1, 'max_depth': 20, 'n_est...",0.851541,0.873631,0.875883,0.731429,0.911873,0.848871,0.061825,5


train score 0.9904536178943112
test score 0.850203133852062


In [29]:
cboost = catboost.CatBoostRegressor(iterations=None,
                        learning_rate=None,
                        depth=None,
                        l2_leaf_reg=None,
                        model_size_reg=None,
                        rsm=None,
                        loss_function='RMSE',
                        border_count=None,
                        feature_border_type=None,
                        per_float_feature_quantization=None,
                        input_borders=None,
                        output_borders=None,
                        fold_permutation_block=None,
                        od_pval=None,
                        od_wait=None,
                        od_type=None,
                        nan_mode=None,
                        counter_calc_method=None,
                        leaf_estimation_iterations=None,
                        leaf_estimation_method=None,
                        thread_count=None,
                        random_seed=98987,
                        use_best_model=None,
                        best_model_min_trees=None,
                        verbose=False,
                        silent=None,
                        logging_level=None,
                        metric_period=None,
                        ctr_leaf_count_limit=None,
                        store_all_simple_ctr=None,
                        max_ctr_complexity=None,
                        has_time=None,
                        allow_const_label=None,
                        one_hot_max_size=None,
                        random_strength=None,
                        name=None,
                        ignored_features=None,
                        train_dir=None,
                        custom_metric=None,
                        eval_metric=None,
                        bagging_temperature=None,
                        save_snapshot=None,
                        snapshot_file=None,
                        snapshot_interval=None,
                        fold_len_multiplier=None,
                        used_ram_limit=None,
                        gpu_ram_part=None,
                        pinned_memory_size=None,
                        allow_writing_files=None,
                        final_ctr_computation_mode=None,
                        approx_on_full_history=None,
                        boosting_type=None,
                        simple_ctr=None,
                        combinations_ctr=None,
                        per_feature_ctr=None,
                        ctr_target_border_count=None,
                        task_type=None,
                        device_config=None,                        
                        devices=None,
                        bootstrap_type=None,
                        subsample=None,                        
                        sampling_unit=None,
                        dev_score_calc_obj_block_size=None,
                        max_depth=None,
                        n_estimators=None,
                        num_boost_round=None,
                        num_trees=None,
                        colsample_bylevel=None,
                        random_state=None,
                        reg_lambda=None,
                        objective=None,
                        eta=None,
                        max_bin=None,
                        gpu_cat_features_storage=None,
                        data_partition=None,
                        metadata=None,
                        early_stopping_rounds=None,
                        cat_features=None,
                        grow_policy=None,
                        min_data_in_leaf=None,
                        min_child_samples=None,
                        max_leaves=None,
                        num_leaves=None,
                        score_function=None,
                        leaf_estimation_backtracking=None,
                        ctr_history_unit=None,
                        monotone_constraints=None)


params = {
    'depth': [1, 3, 5, 10, 20],
    'learning_rate': [0.01, 0.1, 0.5, 1.]
    
    
}


catgrid = sklearn.model_selection.GridSearchCV(cboost, params, scoring=None, n_jobs=None, 
                                                iid='deprecated', refit=True, cv=None, verbose=0, 
                                                pre_dispatch='2*n_jobs', return_train_score=False)

catgrid.fit(oh_train_data, train_labels)

display(pd.DataFrame(catgrid.cv_results_).sort_values('rank_test_score').head())
print('train score', catgrid.score(oh_train_data, train_labels))
print('test score', catgrid.score(oh_test_data, test_labels))

d:\utilites\programming\anaconda-python\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
_catboost.CatBoostError: c:/goagent/pipelines/buildmaster/catboost.git/catboost/private/libs/options/oblivious_tree_options.cpp:117: Maximum tree depth is 16

  FitFailedWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
9,4.318367,0.578589,0.024541,0.003265,5,0.1,"{'depth': 5, 'learning_rate': 0.1}",0.897353,0.910334,0.898616,0.751617,0.942763,0.880137,0.066316,1
5,2.072917,0.161696,0.033910,0.019493,3,0.1,"{'depth': 3, 'learning_rate': 0.1}",0.904240,0.917998,0.897883,0.712303,0.947789,0.876042,0.083655,2
8,4.235502,0.639911,0.027726,0.004820,5,0.01,"{'depth': 5, 'learning_rate': 0.01}",0.869650,0.907819,0.898802,0.770883,0.932913,0.876013,0.056323,3
13,30.910411,0.329409,0.034718,0.020054,10,0.1,"{'depth': 10, 'learning_rate': 0.1}",0.836177,0.863800,0.910513,0.808993,0.923192,0.868535,0.043277,4
12,29.881345,0.394771,0.024942,0.000888,10,0.01,"{'depth': 10, 'learning_rate': 0.01}",0.828901,0.865790,0.904451,0.818219,0.920924,0.867657,0.040351,5


train score 0.9976869549915889
test score 0.8494437893941841
